In [1]:
import pandas as pd
import numpy as np
import math
import requests

liste_med_ban = pd.read_csv('liste_med_ban.csv')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\miniconda\Lib\site-packages\IPython\core\interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\agath\AppData\Local\Temp\ipykernel_18748\3291834190.py", line 6, in <module>
    liste_med_ban = pd.read_csv('liste_med_ban.csv')
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\miniconda\Lib\site-packages\pandas\io\parsers\readers.py", line 1026, in read_csv
    return _read(filepath_or_buffer, kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\miniconda\Lib\site-packages\pandas\io\parsers\readers.py", line 620, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\miniconda\Lib\site-packages\pandas\io\parsers\readers.py", line 1620, in __init__
    self._engine = self._make_engine(f, self.engine)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\miniconda\Lib\site-packa

In [25]:
def cp_et_nom(cp,nom):
    if isinstance(cp,int) and isinstance(nom,str):
        return (str(cp)+' '+nom)
    else:
        return np.nan

In [26]:
liste_med_ban['cp_ville'] = liste_med_ban.apply(lambda x: cp_et_nom(x['codepostalcoordstructure'],x['libellécommunecoordstructure']),axis=1)
liste_med_ban['cp_ville'].iloc[73370]

'94456 Limeil-Brévannes'

In [ ]:
def transfo_adresse(CodePostal,NumeroVoie,NomVoie):
    num = (int(NumeroVoie))
    return (str(num)+', '+str(NomVoie)+', '+str(CodePostal))

liste_med_ban['adresse'] = liste_med_ban.apply(lambda x: transfo_adresse(x['cp_ville'],x['numérovoiecoordstructure'],x['Voie']),axis=1)

In [27]:
API_URL = "https://api-adresse.data.gouv.fr/{}/"

def req_to(endpoint: str, params: dict | None = None, files: dict | None = None) -> dict:
    url = API_URL.format(endpoint)

    if not files:
        response = requests.get(url, params=params)

    else:
        response = requests.post(url, files=files, data=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        raise Exception(f"Request failed with code : {response.status_code}")


def get_geocode(**kwargs) -> dict:
    if not 'q' in kwargs:
        raise ValueError('Il manque un argument obligatoire : q')
    
    return req_to("search", kwargs)


def get_reverse(**kwargs) -> dict:
    if not('lat' in kwargs and 'lon' in kwargs):
        raise ValueError('Il manque des arguments obligatoires : lat et lon')
    
    return req_to("reverse", kwargs)


def get_geocode_csv(file_path: str, **kwargs) -> dict:
    with open(file_path, 'rb') as file:
        files = {'data': file}
        response = req_to("search/csv", kwargs, files) 
    
    return response


def get_reverse_csv(file_path: str) -> dict:
    with open(file_path, 'rb') as file:
        files = {'data': file}
        response = req_to("reverse/csv", files=files) 

    return response

In [28]:
numero=str(int(liste_med_ban['numérovoiecoordstructure'].iloc[0]))
voie = liste_med_ban['Voie'].iloc[0]
cp_ville=liste_med_ban['cp_ville'].iloc[0]
adresse = numero+', '+voie+', '+cp_ville
print(adresse)

# Requête simple directe:
res = get_geocode(q = adresse)
print(res)
#print(type(res['properties']),res['properties'][0]['x'])

52, rue jean jaures, 97110 Pointe-à-Pitre
{'type': 'FeatureCollection', 'version': 'draft', 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-61.536772, 16.23989]}, 'properties': {'label': '52 Rue Jean Jaures 97110 Pointe-à-Pitre', 'score': 0.9688018181818181, 'housenumber': '52', 'id': '97120_0496_00052', 'banId': '3565ae2f-5cbf-4aab-8543-7a3fda9b9059', 'name': '52 Rue Jean Jaures', 'postcode': '97110', 'citycode': '97120', 'x': 656379.35, 'y': 1796029.48, 'city': 'Pointe-à-Pitre', 'context': '971, Guadeloupe', 'type': 'housenumber', 'importance': 0.65682, 'street': 'Rue Jean Jaures'}}], 'attribution': 'BAN', 'licence': 'ETALAB-2.0', 'query': '52, rue jean jaures, 97110 Pointe-à-Pitre', 'limit': 5}


In [29]:
def recup_long(adr):
    if isinstance(adr,str):
        try:
            return(get_geocode(q=adr)['features'][0]['geometry']['coordinates'][0])
        except (IndexError, KeyError, TypeError):
            print(adr)
            return None
    else :
        return (np.nan)

liste_med_ban['longitude API'] = liste_med_ban['adresse'].apply(recup_long)
#liste_med_ban['latitude API'] = get_geocode(q = liste_med_ban['adresse'])['features'][0]['geometry']['coordinates'][1]

#liste_med_ban['adresse','longitude API','latitude API']

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/julie/miniconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/rf/dqdwsk6s06ngm9b4j17fkb2r0000gn/T/ipykernel_98066/3440838157.py", line 17, in <module>
    liste_med_ban['longitude API'] = liste_med_ban['adresse'].apply(recup_long)
                                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/julie/miniconda3/lib/python3.11/site-packages/pandas/core/series.py", line 4924, in apply
    ).apply()
      ^^^^^^^
  File "/Users/julie/miniconda3/lib/python3.11/site-packages/pandas/core/apply.py", line 1427, in apply
    return self.apply_standard()
           ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/julie/miniconda3/lib/python3.11/site-packages/pandas/core/apply.py", line 1507, in apply_standard
    mapped = obj._map_values(
             ^^^^^^^^^^^^^^^^
  File "/Users/julie/miniconda3/lib/python3.11

In [30]:
def distance_long_lat(lat1, lon1, lat2, lon2):
    # Rayon de la Terre en km
    R = 6371.0
    
    # Conversion des degrés en radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    
    # Calcul des différences
    delta_lat = lat2_rad - lat1_rad
    delta_lon = lon2_rad - lon1_rad
    
    # Formule de Haversine
    a = math.sin(delta_lat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(delta_lon / 2)**2
    c = 2 * math.asin(math.sqrt(a))
    
    # Distance
    distance = R * c
    
    return distance

# Exemple d'utilisation
lat1, lon1 = 48.8566, 2.3522  # Paris
lat2, lon2 = 34.0522, -118.2437  # Los Angeles

distance = distance_long_lat(lat1, lon1, lat2, lon2)
print(f"La distance entre Paris et Los Angeles est d'environ {distance:.2f} km.")


La distance entre Paris et Los Angeles est d'environ 9085.51 km.


In [31]:
liste_med_ban['code dépar']

0        971
1        971
2        971
3        971
4        971
        ... 
73369     94
73370     94
73371     93
73372     94
73373     94
Name: code dépar, Length: 73374, dtype: int64

In [32]:
departement_choisi = 39

liste_med_ban_dep=liste_med_ban[liste_med_ban['code dépar']==departement_choisi]
liste_med_ban_dep.shape

(289, 76)

### API OpenRouteService

In [2]:
import openrouteservice

# Clé API OpenRouteService
client = openrouteservice.Client(key='5b3ce3597851110001cf6248fa09b3e419ef40e182777370274818e9')

# Coordonnées (longitude, latitude) pour le départ et l'arrivée
coordinates = [[2.3522, 48.8566], [4.8357, 45.764]]

# Calcul de l'itinéraire
route = client.directions(coordinates, profile='driving-car')

# Temps de trajet en secondes
duration = route['routes'][0]['summary']['duration']
print(f"Temps de trajet : {duration / 60} minutes")

Temps de trajet : 276.305 minutes
